<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/DatosFaltantes_IndicadorAusencia_Sklearn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Agregar un Indicador de ausencia con Scikit-learn ==> MissingIndicator

Scikit-learn tiene una clase **MissingIndicator** para añadir una variable binaria que marque los valores ausentes.

MissingIndicator tiene la opción de añadir un indicador binario (variable) para cada variable en un conjunto de datos o solamente para aquellas que tienen NA en el segmento de entrenamiento.

### Atención!

El transformer solo devuelve las variables binarias, que luego deben ser añadidas a los datos originales de entrenamiento.

### Más detalles acerca de los transformadores

- [MissingIndicaror](https://scikit-learn.org/stable/modules/generated/sklearn.impute.MissingIndicator.html#sklearn.impute.MissingIndicator)

## En este demo:

Vamos a añadir un Indicador de ausencia (Missing Indicator) para las variables en el Ames House Price Dataset

- Para bajar los datos, por favor referirse a la clase **Datasets** en la  **Sección 1** del curso.

In [ ]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

# estas son las clases para sustitutición con sklearn
from sklearn.impute import SimpleImputer, MissingIndicator
from sklearn.pipeline import Pipeline

# separar segmentos prueba/entrenamiento
from sklearn.model_selection import train_test_split

In [ ]:
# solo usaremos las siguientes variables categóricas en el demo:

# una mezcla de variables categóricas y numéricas

cols_to_use = ['BsmtQual', 'FireplaceQu', 'MSZoning',
               'BsmtUnfSF', 'LotFrontage', 'MasVnrArea',
               'Street', 'Alley', 'SalePrice']

In [ ]:
# carguemos los datos 
data = pd.read_csv('../houseprice.csv', usecols=cols_to_use)
print(data.shape)
data.head()

(1460, 9)


,MSZoning,LotFrontage,Street,Alley,MasVnrArea,BsmtQual,BsmtUnfSF,FireplaceQu,SalePrice
0,RL,65.0,Pave,NaN,196.0,Gd,150,NaN,208500
1,RL,80.0,Pave,NaN,0.0,Gd,284,TA,181500
2,RL,68.0,Pave,NaN,162.0,Gd,434,TA,223500
3,RL,60.0,Pave,NaN,0.0,TA,540,Gd,140000
4,RL,84.0,Pave,NaN,350.0,Gd,490,TA,250000


In [ ]:
# revisemos los valores nulos
data.isnull().mean()

MSZoning       0.000000
LotFrontage    0.177397
Street         0.000000
Alley          0.937671
MasVnrArea     0.005479
BsmtQual       0.025342
BsmtUnfSF      0.000000
FireplaceQu    0.472603
SalePrice      0.000000
dtype: float64

In [ ]:
# separar datos en segmentos entrenamiento y prueba

# primero, separemos el target (SalePrice) del resto de las variables (features)

cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use], # solo las variables
                                                    data['SalePrice'], # el target
                                                    test_size=0.3, # el porcentaje de obs en el segmento de prueba
                                                    random_state=0) # para reproducir
X_train.shape, X_test.shape

((1022, 8), (438, 8))

In [ ]:
# evaluemos el porcentaje de datos ausentes nuevamente
X_train.isnull().mean()

BsmtQual       0.023483
FireplaceQu    0.467710
MSZoning       0.000000
BsmtUnfSF      0.000000
LotFrontage    0.184932
MasVnrArea     0.004892
Street         0.000000
Alley          0.939335
dtype: float64

## Añadir un Indicador  de Ausencia (Missing Indicator)

In [ ]:
indicator = MissingIndicator(error_on_new=True, features='missing-only')
indicator.fit(X_train)  

MissingIndicator(error_on_new=True, features='missing-only', missing_values=nan,
                 sparse='auto')

In [ ]:
# podemos ver las variables con valores nulos na:
# el resultado muestra el índice (index)

indicator.features_

array([0, 1, 4, 5, 7], dtype=int64)

In [ ]:
# podemos encontrar el nombre de las variables pasando el índice 
# a la lista de columnas
X_train.columns[indicator.features_]

Index(['BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'Alley'], dtype='object')

In [ ]:
# el 'indicator' devuelve solamente los indicadores adicionales
# cuando tranformamos los datos

tmp = indicator.transform(X_train)

tmp

array([[False,  True,  True, False,  True],
       [False, False,  True, False,  True],
       [False,  True, False, False,  True],
       ...,
       [ True,  True, False, False,  True],
       [False, False,  True, False,  True],
       [False,  True, False, False,  True]])

In [ ]:
# ahora necesitamos unirlo manualmente al segmento X_train

# creemos una columna por cada uno de los nuevos indicadores MissingIndicators
indicator_cols = [c+'_NA' for c in X_train.columns[indicator.features_]]

# y ahora concatenamos
X_train = pd.concat([
    X_train.reset_index(),
    pd.DataFrame(tmp, columns = indicator_cols)],
    axis=1)

X_train.head()

,index,BsmtQual,FireplaceQu,MSZoning,BsmtUnfSF,LotFrontage,MasVnrArea,Street,Alley,BsmtQual_NA,FireplaceQu_NA,LotFrontage_NA,MasVnrArea_NA,Alley_NA
0,64,Gd,NaN,RL,318,NaN,573.0,Pave,NaN,False,True,True,False,True
1,682,Gd,Gd,RL,288,NaN,0.0,Pave,NaN,False,False,True,False,True
2,960,TA,NaN,RL,162,50.0,0.0,Pave,NaN,False,True,False,False,True
3,1384,TA,NaN,RL,356,60.0,0.0,Pave,NaN,False,True,False,False,True
4,1100,TA,NaN,RL,0,60.0,0.0,Pave,NaN,False,True,False,False,True


In [ ]:
# repetimos para el segmento de prueba
tmp = indicator.transform(X_test)

X_test = pd.concat([
    X_test.reset_index(),
    pd.DataFrame(tmp, columns = indicator_cols)],
    axis=1)

X_test.head()

,index,BsmtQual,FireplaceQu,MSZoning,BsmtUnfSF,LotFrontage,MasVnrArea,Street,Alley,BsmtQual_NA,FireplaceQu_NA,LotFrontage_NA,MasVnrArea_NA,Alley_NA
0,529,TA,TA,RL,816,NaN,NaN,Pave,NaN,False,False,True,True,True
1,491,TA,TA,RL,238,79.0,0.0,Pave,NaN,False,False,False,False,True
2,459,TA,TA,RL,524,NaN,161.0,Pave,NaN,False,False,True,False,True
3,279,Gd,TA,RL,768,83.0,299.0,Pave,NaN,False,False,False,False,True
4,655,TA,NaN,RM,525,21.0,381.0,Pave,NaN,False,True,False,False,True


### SimpleImputer en un conjunto de datos diferente

In [ ]:
# Ahora sustituimos los valores ausentes con el SimpleImputer

# creamos una instancia del SimpleImputer
# indicamos que queramos sustituir con la 
# categoría más frecuente
imputer = SimpleImputer(strategy='most_frequent')

# ajustamos el 'imputer' al set de entrenamiento asi aprende
# la moda
imputer.fit(X_train)

SimpleImputer(add_indicator=False, copy=True, fill_value=None,
              missing_values=nan, strategy='most_frequent', verbose=0)

In [ ]:
# podemos ver cuales fueron los valores frecuentes aprendidos:
imputer.statistics_

array([0, 'TA', 'Gd', 'RL', 0, 60.0, 0.0, 'Pave', 'Pave', False, False,
       False, False, True], dtype=object)

**Nota** el transformer aprende cual es el valor de la categoría más frecuente para AMBAS variables las categóricas y las numéricas.

In [ ]:
# y ahora sustituimos ambos segmentos de prueba y entrenamiento

# NOTA: los datos se devuelven como un numpy array!!!
X_train = imputer.transform(X_train)
X_test = imputer.transform(X_test)

X_train

array([[64, 'Gd', 'Gd', ..., True, False, True],
       [682, 'Gd', 'Gd', ..., True, False, True],
       [960, 'TA', 'Gd', ..., False, False, True],
       ...,
       [1216, 'TA', 'Gd', ..., False, False, True],
       [559, 'Gd', 'TA', ..., True, False, True],
       [684, 'Gd', 'Gd', ..., False, False, True]], dtype=object)